In [43]:
import pandas as pd

In [44]:
df = pd.read_csv('/content/Conversation.csv')
df

,Unnamed: 0,question,answer
0,0,"hi, how are you doing?",i'm fine. how about yourself?
1,1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,3,no problem. so how have you been?,i've been great. what about you?
4,4,i've been great. what about you?,i've been good. i'm in school right now.
...,...,...,...
3720,3720,that's a good question. maybe it's not old age.,are you right-handed?
3721,3721,are you right-handed?,yes. all my life.
3722,3722,yes. all my life.,you're wearing out your right hand. stop using...
3723,3723,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

In [45]:
!pip install transformers datasets torch

In [46]:
import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
import evaluate

In [47]:
!pip install sentencepiece

In [48]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'status' is the label column
label_encoder = LabelEncoder()
df['labels'] = label_encoder.fit_transform(df['answer'])

# Print the encoded labels to check
print(df[['answer', 'labels']])

                                                 answer  labels
0                         i'm fine. how about yourself?    1303
1                   i'm pretty good. thanks for asking.    1347
2                     no problem. so how have you been?    1783
3                      i've been great. what about you?    1377
4              i've been good. i'm in school right now.    1376
...                                                 ...     ...
3720                              are you right-handed?     156
3721                                  yes. all my life.    3314
3722  you're wearing out your right hand. stop using...    3496
3723        but i do all my writing with my right hand.     223
3724  start typing instead. that way your left hand ...    2273

[3725 rows x 2 columns]


In [49]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq

# Load your dataset
dataset = load_dataset('csv', data_files='/content/Conversation.csv')

# Split the dataset into training and testing sets
train_test_split = dataset['train'].train_test_split(test_size=0.2)

# Access the training and testing datasets
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [50]:
# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-small')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [51]:
# Tokenization function
def tokenize_function(examples):
    inputs = tokenizer(examples['question'], padding='max_length', truncation=True, return_tensors="pt")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['answer'], padding='max_length', truncation=True, return_tensors="pt")
    inputs['labels'] = labels['input_ids']
    return inputs

# Tokenize the datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2980 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/745 [00:00<?, ? examples/s]

In [53]:
# Load the model
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-small')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy='epoch',     # evaluation strategy
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    num_train_epochs=9,              # number of training epochs
    weight_decay=0.01,               # strength of weight decay
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [54]:
# Set up data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,  # Use the tokenized train dataset
    eval_dataset=tokenized_test_dataset,     # Use the tokenized test dataset
    data_collator=data_collator,
)



In [55]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.882113
2,9.445500,0.152480
3,0.469600,0.058420
4,0.469600,0.052271


Epoch,Training Loss,Validation Loss
1,No log,1.882113
2,9.445500,0.152480
3,0.469600,0.058420
4,0.469600,0.052271
5,0.078900,0.051008
6,0.062000,0.050459
7,0.058900,0.050133
8,0.058900,0.049967
9,0.057300,0.049915


TrainOutput(global_step=3357, training_loss=1.5211163510301833, metrics={'train_runtime': 2777.1556, 'train_samples_per_second': 9.657, 'train_steps_per_second': 1.209, 'total_flos': 4985583336161280.0, 'train_loss': 1.5211163510301833, 'epoch': 9.0})

In [56]:
# Evaluate the model
results = trainer.evaluate()
print(f"Evaluation Results: {results}")

Evaluation Results: {'eval_loss': 0.04991540685296059, 'eval_runtime': 22.885, 'eval_samples_per_second': 32.554, 'eval_steps_per_second': 4.108, 'epoch': 9.0}


In [59]:
from transformers import AutoModelForSeq2SeqLM

# Load the original pre-trained model
original_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

# Load the fine-tuned model
fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained("/content/results/checkpoint-1119")

# Compare weights of a specific layer
original_weights = original_model.encoder.block[0].layer[0].SelfAttention.q.weight
fine_tuned_weights = fine_tuned_model.encoder.block[0].layer[0].SelfAttention.q.weight

# Check if weights are different
if not torch.equal(original_weights, fine_tuned_weights):
    print("The model has been fine-tuned.")
else:
    print("The model weights have not changed (no fine-tuning).")


The model has been fine-tuned.


In [60]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the fine-tuned model
model = T5ForConditionalGeneration.from_pretrained('/content/results/checkpoint-1119')  # Path to your fine-tuned model directory
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-small')

# Function to generate response using the fine-tuned model
def generate_response(input_text):
    # Tokenize the input
    inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)

    # Generate the output using the model
    output_sequences = model.generate(input_ids=inputs['input_ids'], max_length=50)

    # Decode the generated sequence
    response = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    return response

# Example usage: Chatbot interaction
user_input = "Hi, how are you?"
response = generate_response(user_input)

print(f"User: {user_input}")
print(f"Bot: {response}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


User: Hi, how are you?
Bot: i'm going to go to the movies.
